<a href="https://colab.research.google.com/github/TheAcoff/chat-Fismat/blob/main/Chat_Fismat_Obsidian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ferramenta de Auxílio em Estudos

Para este código, iremos utilizar dois modelos diferentes. Um para a detecção de intenção do usuário, tendo como saída variáveis precisas que irão entrar em uma lista para serem input de um segundo modelo que terá os dados treinados para a Física Matemática.

Para o modelo1, temos a saída:
$$
output1 = \begin{pmatrix} intenção \\ tema \\ ordem \end{pmatrix}  = input2
$$




In [1]:
import google.generativeai as genai

#Configuração de API
APIKey = 'APIKEY'
genai.configure(api_key=APIKey)

# Definição de Parâmetros
Nessa parte, definimos os parâmetros dos dois modelos. Seja $n$ o número do modelo, podendo ser $0$ ou $1$ para os parâmetros, então, definimos previamente: generation_config$\,n$ , safety_settings$\,n$

## Parâmetros do Modelo 0

In [5]:
generation_config0 = {
  "temperature": 0.4,
  "top_p": 0,
  "top_k": 0,
  "max_output_tokens": 2048
}

safety_settings0 = {
    'HARASSMENT':'BLOCK_MEDIUM_AND_ABOVE',
    'HATE':'BLOCK_MEDIUM_AND_ABOVE',
    'SEXUAL':'BLOCK_MEDIUM_AND_ABOVE',
    'DANGEROUS':'BLOCK_MEDIUM_AND_ABOVE',
}

## Parâmetros do Modelo 1

In [3]:
generation_config1 = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 12000
}

safety_settings1 = {
    'HARASSMENT':'BLOCK_NONE',
    'HATE':'BLOCK_NONE',
    'SEXUAL':'BLOCK_NONE',
    'DANGEROUS':'BLOCK_NONE',
}

# Incializando os Modelos

In [6]:
model0 = model = genai.GenerativeModel(
    model_name='gemini-1.0-pro',
    generation_config = generation_config0,
    safety_settings = safety_settings0,
    )

#model1 = model = genai.GenerativeModel(
#    model_name='gemini-1.5-pro-latest',
#    generation_config = generation_config1,
#    safety_settings = safety_settings1,
#    )

In [38]:
def string_para_lista(texto):
  # Remove as chaves e divide a string em elementos usando a vírgula como separador
  elementos = texto.strip("[]").split(",")

  elementos = [elemento for elemento in elementos]

  return elementos


In [ ]:
prompt = str(input('Input: '))

chat0 = model0.start_chat(history=[
  {
    "role": "user",
    "parts": ["Aja como um identificador de intenção e ordem e me ajude a detectar o tema. \nDevemos ter dois resultados possíveis de Intenção desejadas pelo usuário: Dúvida e Edição. \nJá para ordem: Nenhuma, Resuma, Detalhe, Crie.\nPara isso, responda no seguinte formato, como se fosse declarar uma matriz em Python, sem utilizar os espaços: [Intenção,Ordem,Tema] \n---\nExemplo1: Gostaria de saber mais sobre Inteligência Artificial \n[Dúvida,Detalhe,Inteligência Artificial]\nExemplo2: Me ajude com minhas notas sobre Redes Neurais \n[Edição,Complemente,Redes Neurais] \nExemplo3: Crie uma nota sobre Produto Interno\n[Geração,Crie,Produto Interno]\nExemplo4: Me explique sobre Espaço Vetorial. \n[Dúvida,Nenhma,Espaço Vetorial] \nExemplo 5: Fale mais sobre Jupiter e suas luas \n[Dúvida,Detalhe,Luas de Júpiter] \nExemplo 6: Resume sobre crateras da Terra \n[Dúvida,Resuma,Crateras da Terra] \nExemplo7: Resuma minhas notas sobre Teclado \n[Edição,Resuma,Teclado]"]
  },
  {
    "role": "model",
    "parts": ["Ok, irei analisar as frases e fazer no modelo [Intenção,Ordem,Tema]"]
  }
])

while prompt != 'fim':
  #Envia o prompt e converte a saída para uma matriz.
  output0 = chat0.send_message(prompt)
  output0M = string_para_lista(output0.text)

  #Separa as variáveis da saída
  Intencao = output0M[0]
  Ordem = output0M[1]
  Tema = output0M[2]
  print(output0M)



  prompt = str(input('Input: '))